In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '954438', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41846,CN0100100000000,1,1,"Autauga County, AL",Jan-21(p),"25,757","24,904",853,3.3,01001
41847,CN0100300000000,1,3,"Baldwin County, AL",Jan-21(p),"95,885","92,512","3,373",3.5,01003
41848,CN0100500000000,1,5,"Barbour County, AL",Jan-21(p),"8,391","7,826",565,6.7,01005
41849,CN0100700000000,1,7,"Bibb County, AL",Jan-21(p),"8,622","8,270",352,4.1,01007
41850,CN0100900000000,1,9,"Blount County, AL",Jan-21(p),"24,792","24,190",602,2.4,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
45060,CN7214500000000,72,145,"Vega Baja Municipio, PR",Jan-21(p),"12,691","11,248",1443,11.4,72145
45061,CN7214700000000,72,147,"Vieques Municipio, PR",Jan-21(p),"2,698","2,368",330,12.2,72147
45062,CN7214900000000,72,149,"Villalba Municipio, PR",Jan-21(p),"6,846","5,876",970,14.2,72149
45063,CN7215100000000,72,151,"Yabucoa Municipio, PR",Jan-21(p),"8,268","7,191",1077,13.0,72151
45064,CN7215300000000,72,153,"Yauco Municipio, PR",Jan-21(p),"9,000","7,770",1230,13.7,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1183264,2021-04-02,Webster,West Virginia,54101.0,449,7.0
1183265,2021-04-02,Wetzel,West Virginia,54103.0,1180,26.0
1183266,2021-04-02,Wirt,West Virginia,54105.0,370,3.0
1183267,2021-04-02,Wood,West Virginia,54107.0,7460,164.0
1183268,2021-04-02,Wyoming,West Virginia,54109.0,1835,38.0
1183269,2021-04-02,Adams,Wisconsin,55001.0,1800,15.0
1183270,2021-04-02,Ashland,Wisconsin,55003.0,1283,16.0
1183271,2021-04-02,Barron,Wisconsin,55005.0,5686,76.0
1183272,2021-04-02,Bayfield,Wisconsin,55007.0,1186,19.0
1183273,2021-04-02,Brown,Wisconsin,55009.0,33859,272.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
391,03/26/2021,3106,1139,264,54,10,2951,4108,260,51,56,551,185,60,9,0,494,685,52,8,8,887,313,72,18,0,907,1246,83,16,17,451,189,35,4,0,420,613,31,8,8,945,375,73,17,0,897,1249,76,15,15,272,67,24,6,0,233,311,18,4,4,11000
392,03/27/2021,1994,908,234,41,18,2942,4092,257,50,57,342,149,53,15,0,497,690,52,10,10,548,258,71,13,1,898,1232,81,16,16,249,169,34,5,2,418,615,32,7,8,670,242,67,7,0,892,1233,75,14,14,185,75,9,1,0,238,316,17,3,3,11000
393,03/28/2021,1525,793,169,51,9,2877,4013,246,50,59,240,126,37,10,0,493,685,53,10,10,497,218,48,14,0,873,1201,76,16,17,192,166,19,6,0,413,613,30,6,7,478,216,52,16,0,866,1199,71,14,14,118,58,13,5,0,231,308,16,4,4,11000
394,03/29/2021,3104,1251,225,35,16,2791,3913,234,48,58,569,214,48,11,2,486,675,52,10,10,872,338,59,10,0,834,1148,72,15,15,418,208,37,4,0,395,590,29,6,7,959,363,69,9,0,848,1184,67,14,14,286,114,12,1,0,228,308,16,3,3,11000
395,03/30/2021,2411,1026,106,20,16,2701,3798,214,43,56,444,174,36,2,1,472,659,49,9,10,706,321,25,8,1,794,1107,64,14,14,359,204,12,1,0,384,580,26,5,5,689,266,25,9,0,822,1139,60,13,13,212,46,8,0,0,228,303,15,3,3,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
391,03/26/2021,551,9,New York,Bronx,36005.0
392,03/27/2021,342,15,New York,Bronx,36005.0
393,03/28/2021,240,10,New York,Bronx,36005.0
394,03/29/2021,569,11,New York,Bronx,36005.0
395,03/30/2021,444,2,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
391,03/26/2021,887,18,Kings,New York,36047.0
392,03/27/2021,548,13,Kings,New York,36047.0
393,03/28/2021,497,14,Kings,New York,36047.0
394,03/29/2021,872,10,Kings,New York,36047.0
395,03/30/2021,706,8,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,8971,157


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
382,03/17/2021,929,14,New York,Queens,36081.0,2021-03-17
383,03/18/2021,877,15,New York,Queens,36081.0,2021-03-18
384,03/19/2021,863,14,New York,Queens,36081.0,2021-03-19
385,03/20/2021,709,16,New York,Queens,36081.0,2021-03-20
386,03/21/2021,655,16,New York,Queens,36081.0,2021-03-21
387,03/22/2021,1084,10,New York,Queens,36081.0,2021-03-22
388,03/23/2021,871,15,New York,Queens,36081.0,2021-03-23
389,03/24/2021,969,15,New York,Queens,36081.0,2021-03-24
390,03/25/2021,1047,17,New York,Queens,36081.0,2021-03-25
391,03/26/2021,945,17,New York,Queens,36081.0,2021-03-26


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
346,02/09/2021,Richmond,New York,36085.0,244,3.0,2021-02-09
347,02/10/2021,Richmond,New York,36085.0,272,5.0,2021-02-10
348,02/11/2021,Richmond,New York,36085.0,227,5.0,2021-02-11
349,02/12/2021,Richmond,New York,36085.0,232,6.0,2021-02-12
350,02/13/2021,Richmond,New York,36085.0,136,5.0,2021-02-13
351,02/14/2021,Richmond,New York,36085.0,113,3.0,2021-02-14
352,02/15/2021,Richmond,New York,36085.0,232,5.0,2021-02-15
353,02/16/2021,Richmond,New York,36085.0,189,3.0,2021-02-16
354,02/17/2021,Richmond,New York,36085.0,196,3.0,2021-02-17
355,02/18/2021,Richmond,New York,36085.0,110,2.0,2021-02-18


In [46]:
print (data_date_dt)

2021-04-03 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1180117,1001.0,99.0
1180118,1003.0,301.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,7912


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1183119,51690.0,74.0
1183120,51115.0,12.0
1183121,51117.0,62.0
1183122,51119.0,19.0
1183123,51121.0,86.0
1183124,51125.0,12.0
1183125,51127.0,14.0
1183126,51700.0,208.0
1183127,51710.0,235.0
1183128,51131.0,34.0


In [50]:
covid_death.shape

(3252, 2)

In [51]:
print (County_covid_death)

{1001.0: 99.0, 1003.0: 301.0, 1005.0: 55.0, 1007.0: 58.0, 1009.0: 132.0, 1011.0: 40.0, 1013.0: 66.0, 1015.0: 310.0, 1017.0: 119.0, 1019.0: 43.0, 1021.0: 109.0, 1023.0: 24.0, 1025.0: 60.0, 1027.0: 54.0, 1029.0: 41.0, 1031.0: 111.0, 1033.0: 126.0, 1035.0: 27.0, 1037.0: 24.0, 1039.0: 111.0, 1041.0: 57.0, 1043.0: 186.0, 1045.0: 109.0, 1047.0: 149.0, 1049.0: 180.0, 1051.0: 202.0, 1053.0: 74.0, 1055.0: 344.0, 1057.0: 59.0, 1059.0: 82.0, 1061.0: 74.0, 1063.0: 34.0, 1065.0: 73.0, 1067.0: 43.0, 1069.0: 275.0, 1071.0: 107.0, 1073.0: 1466.0, 1075.0: 33.0, 1077.0: 223.0, 1079.0: 94.0, 1081.0: 166.0, 1083.0: 146.0, 1085.0: 53.0, 1087.0: 47.0, 1089.0: 493.0, 1091.0: 58.0, 1093.0: 101.0, 1095.0: 217.0, 1097.0: 792.0, 1099.0: 40.0, 1101.0: 558.0, 1103.0: 265.0, 1105.0: 27.0, 1107.0: 57.0, 1109.0: 74.0, 1111.0: 41.0, 1113.0: 37.0, 1117.0: 235.0, 1115.0: 234.0, 1119.0: 32.0, 1121.0: 168.0, 1123.0: 146.0, 1125.0: 437.0, 1127.0: 274.0, 1129.0: 38.0, 1131.0: 26.0, 1133.0: 71.0, 2013.0: 2.0, 2016.0: 0.0, 20

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1138215,2021-03-20,New Haven,Connecticut,9009.0,76466,1980.0,2021-03-20,-14 days,NaN
1141461,2021-03-21,New Haven,Connecticut,9009.0,76466,1980.0,2021-03-21,-13 days,0.0
1144708,2021-03-22,New Haven,Connecticut,9009.0,77335,1983.0,2021-03-22,-12 days,869.0
1147955,2021-03-23,New Haven,Connecticut,9009.0,77638,1986.0,2021-03-23,-11 days,303.0
1151202,2021-03-24,New Haven,Connecticut,9009.0,78094,1988.0,2021-03-24,-10 days,456.0
1154449,2021-03-25,New Haven,Connecticut,9009.0,78536,1992.0,2021-03-25,-9 days,442.0
1157696,2021-03-26,New Haven,Connecticut,9009.0,79060,1992.0,2021-03-26,-8 days,524.0
1160944,2021-03-27,New Haven,Connecticut,9009.0,79060,1992.0,2021-03-27,-7 days,0.0
1164192,2021-03-28,New Haven,Connecticut,9009.0,79060,1992.0,2021-03-28,-6 days,0.0
1167440,2021-03-29,New Haven,Connecticut,9009.0,80047,2000.0,2021-03-29,-5 days,987.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,5327.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
346,02/09/2021,Richmond,New York,36085.0,244,3.0,2021-02-09
347,02/10/2021,Richmond,New York,36085.0,272,5.0,2021-02-10
348,02/11/2021,Richmond,New York,36085.0,227,5.0,2021-02-11
349,02/12/2021,Richmond,New York,36085.0,232,6.0,2021-02-12
350,02/13/2021,Richmond,New York,36085.0,136,5.0,2021-02-13
351,02/14/2021,Richmond,New York,36085.0,113,3.0,2021-02-14
352,02/15/2021,Richmond,New York,36085.0,232,5.0,2021-02-15
353,02/16/2021,Richmond,New York,36085.0,189,3.0,2021-02-16
354,02/17/2021,Richmond,New York,36085.0,196,3.0,2021-02-17
355,02/18/2021,Richmond,New York,36085.0,110,2.0,2021-02-18


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,19.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-04-03 12:41:46.300777
4


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 99.0
1003.0 301.0
1005.0 55.0
1007.0 58.0
1009.0 132.0
1011.0 40.0
1013.0 66.0
1015.0 310.0
1017.0 119.0
1019.0 43.0
1021.0 109.0
1023.0 24.0
1025.0 60.0
1027.0 54.0
1029.0 41.0
1031.0 111.0
1033.0 126.0
1035.0 27.0
1037.0 24.0
1039.0 111.0
1041.0 57.0
1043.0 186.0
1045.0 109.0
1047.0 149.0
1049.0 180.0
1051.0 202.0
1053.0 74.0
1055.0 344.0
1057.0 59.0
1059.0 82.0
1061.0 74.0
1063.0 34.0
1065.0 73.0
1067.0 43.0
1069.0 275.0
1071.0 107.0
1073.0 1466.0
1075.0 33.0
1077.0 223.0
1079.0 94.0
1081.0 166.0
1083.0 146.0
1085.0 53.0
1087.0 47.0
1089.0 493.0
1091.0 58.0
1093.0 101.0
1095.0 217.0
1097.0 792.0
1099.0 40.0
1101.0 558.0
1103.0 265.0
1105.0 27.0
1107.0 57.0
1109.0 74.0
1111.0 41.0
1113.0 37.0
1117.0 235.0
1115.0 234.0
1119.0 32.0
1121.0 168.0
1123.0 146.0
1125.0 437.0
1127.0 274.0
1129.0 38.0
1131.0 26.0
1133.0 71.0
2013.0 2.0
2016.0 0.0
2020.0 161.0
2050.0 20.0
2997.0 0.0
2068.0 0.0
2070.0 1.0
2090.0 32.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 5.0
2158.0 3.0
2170

24037.0 123.0
24041.0 36.0
nan 58.0
24043.0 263.0
24045.0 150.0
24047.0 95.0
25001.0 445.0
25003.0 277.0
25005.0 1645.0
25007.0 0.0
25009.0 2300.0
25011.0 108.0
25013.0 1453.0
25015.0 284.0
25017.0 3661.0
25019.0 0.0
25021.0 1731.0
25023.0 1382.0
25025.0 1794.0
nan 12.0
25027.0 2157.0
26001.0 25.0
26003.0 5.0
26005.0 118.0
26007.0 49.0
26009.0 20.0
26011.0 27.0
26013.0 34.0
26015.0 52.0
26017.0 307.0
26019.0 32.0
26021.0 251.0
26023.0 112.0
26025.0 246.0
26027.0 68.0
26029.0 13.0
26031.0 38.0
26033.0 46.0
26035.0 69.0
26037.0 75.0
26039.0 6.0
26041.0 86.0
26043.0 71.0
26045.0 162.0
26047.0 40.0
26049.0 783.0
26051.0 43.0
26053.0 50.0
26055.0 77.0
26057.0 109.0
26059.0 79.0
26061.0 40.0
26063.0 62.0
26065.0 303.0
26067.0 79.0
26069.0 67.0
26071.0 51.0
26073.0 77.0
26075.0 264.0
26077.0 328.0
26079.0 16.0
26081.0 748.0
26083.0 2.0
26085.0 12.0
26087.0 144.0
26089.0 16.0
26091.0 139.0
26093.0 157.0
26095.0 2.0
26097.0 3.0
26099.0 2041.0
26101.0 23.0
26103.0 57.0
26105.0 27.0
26107.0 23.0


47003.0 125.0
47005.0 40.0
47007.0 11.0
47009.0 194.0
47011.0 145.0
47013.0 61.0
47015.0 31.0
47017.0 82.0
47019.0 156.0
47021.0 50.0
47023.0 48.0
47025.0 74.0
47027.0 31.0
47029.0 98.0
47031.0 120.0
47033.0 48.0
47035.0 125.0
47037.0 902.0
47041.0 53.0
47039.0 38.0
47043.0 107.0
47045.0 103.0
47047.0 78.0
47049.0 45.0
47051.0 87.0
47053.0 143.0
47055.0 98.0
47057.0 48.0
47059.0 151.0
47061.0 31.0
47063.0 170.0
47065.0 478.0
47067.0 12.0
47069.0 63.0
47071.0 66.0
47073.0 104.0
47075.0 60.0
47077.0 74.0
47079.0 75.0
47081.0 43.0
47083.0 33.0
47085.0 21.0
47087.0 35.0
47089.0 121.0
47091.0 38.0
47093.0 617.0
47095.0 26.0
47097.0 44.0
47099.0 86.0
47101.0 25.0
47103.0 63.0
47105.0 68.0
47111.0 74.0
47113.0 238.0
47115.0 46.0
47117.0 57.0
47119.0 162.0
47107.0 96.0
47109.0 54.0
47121.0 23.0
47123.0 95.0
47125.0 221.0
47127.0 17.0
47129.0 39.0
47131.0 96.0
47133.0 60.0
47135.0 28.0
47137.0 24.0
47139.0 24.0
47141.0 173.0
47143.0 75.0
47145.0 100.0
47147.0 127.0
47149.0 409.0
47151.0 45.0
47

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,8971,157


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 8971}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 34.0
1001 96.0
1009 73.0
1013 35.0
1015 78.0
1017 23.0
1031 49.0
1033 58.0
1039 19.0
1043 111.0
1045 44.0
1051 158.0
1055 53.0
1067 17.0
1069 97.0
1071 39.0
1077 129.0
1079 23.0
1083 94.0
1089 377.0
1095 79.0
1097 373.0
1103 84.0
1111 25.0
1113 49.0
1115 95.0
1117 308.0
1121 152.0
2261 39.0
4021 932.0
5009 32.0
5011 15.0
5033 57.0
5037 13.0
5045 134.0
5047 15.0
5051 64.0
5053 11.0
5063 17.0
5083 3.0
5085 62.0
5087 15.0
5115 44.0
5117 2.0
5121 17.0
5125 80.0
5131 134.0
5145 75.0
6007 137.0
6017 119.0
6023 80.0
6027 19.0
6061 602.0
6079 358.0
6089 304.0
6093 35.0
8019 18.0
8039 61.0
8047 5.0
8051 34.0
8057 0.0
8065 17.0
8067 158.0
8071 16.0
8075 61.0
8077 113.0
8083 37.0
8085 30.0
8093 30.0
8099 2.0
8101 466.0
8103 26.0
8119 79.0
8121 8.0
9005 1122.0
9007 612.0
9011 807.0
9015 362.0
10001 623.0
12019 296.0
12033 389.0
12089 81.0
12101 1397.0
12109 613.0
12113 215.0
13015 335.0
13039 65.0
13045 213.0
13047 162.0
13057 898.0
13073 187.0
13077 172.0
13085 51.0
13113 147.0
13115 241.0


42031 76.0
42033 465.0
42035 162.0
42037 143.0
42039 102.0
42041 620.0
42047 169.0
42051 364.0
42055 481.0
42057 37.0
42063 154.0
42065 97.0
42067 60.0
42073 296.0
42075 520.0
42077 1840.0
42081 482.0
42083 81.0
42085 166.0
42089 920.0
42095 1946.0
42099 159.0
42103 305.0
42105 14.0
42107 488.0
42109 89.0
42111 133.0
42115 191.0
42117 115.0
42121 68.0
42123 42.0
42125 809.0
42127 153.0
42129 1301.0
42131 183.0
42133 1939.0
44001 187.0
44005 314.0
44009 528.0
45003 204.0
45007 541.0
45015 504.0
45023 72.0
45035 488.0
45045 2587.0
45055 138.0
45057 204.0
45063 688.0
45073 244.0
45077 492.0
45083 1397.0
45091 770.0
46015 4.0
46019 5.0
46025 28.0
46029 88.0
46039 8.0
46043 11.0
46045 5.0
46051 16.0
46053 8.0
46055 0.0
46059 0.0
46063 0.0
46067 46.0
46077 72.0
46079 50.0
46081 18.0
46093 25.0
46097 10.0
46101 4.0
46105 4.0
46111 4.0
46115 8.0
46117 1.0
46119 1.0
46123 13.0
46125 21.0
46129 18.0
47001 135.0
47003 221.0
47009 219.0
47011 412.0
47015 30.0
47019 149.0
47021 153.0
47025 52.0
470

39001 35.0
39031 41.0
39041 485.0
39047 26.0
39049 2933.0
39079 60.0
39087 52.0
39097 57.0
39101 110.0
39115 6.0
39129 100.0
39131 51.0
39173 315.0
40003 5.0
40007 6.0
40009 29.0
40011 4.0
40025 6.0
40029 3.0
40039 14.0
40041 49.0
40043 5.0
40057 13.0
40063 24.0
40069 15.0
40073 19.0
40077 1.0
40079 47.0
40097 42.0
40099 15.0
40101 57.0
40109 937.0
40111 19.0
40115 81.0
40121 43.0
40127 6.0
40143 679.0
40151 5.0
41001 67.0
41003 105.0
41005 498.0
41013 22.0
41015 43.0
41019 142.0
41025 11.0
41027 9.0
41031 16.0
41039 282.0
41041 42.0
41057 58.0
41063 5.0
41067 547.0
42011 2141.0
42021 276.0
42023 12.0
42043 828.0
42049 454.0
42059 123.0
42061 105.0
42069 881.0
42071 2107.0
42079 1184.0
42087 120.0
42091 2928.0
42093 43.0
42097 266.0
42113 8.0
42119 90.0
44003 617.0
45017 8.0
45019 963.0
45021 137.0
45037 48.0
45039 55.0
45041 162.0
45043 84.0
45047 215.0
45051 843.0
45059 264.0
45071 94.0
46003 4.0
46009 6.0
46013 84.0
46021 0.0
46023 25.0
46033 13.0
46035 95.0
46037 4.0
46047 8.0
4605

48449 14.0
48457 27.0
48471 185.0
48473 134.0
48495 16.0
49039 38.0
51029 17.0
51081 43.0
51153 1218.0
51590 55.0
51620 30.0
51660 169.0
51683 59.0
51685 22.0
51690 24.0
51700 562.0
51740 248.0
51760 481.0
53025 199.0
54013 9.0
54109 49.0
1065 21.0
2013 0.0
2185 19.0
4009 25.0
4012 21.0
5035 46.0
5095 7.0
5099 6.0
6039 146.0
6071 1589.0
6075 450.0
6077 1403.0
6099 1131.0
12013 14.0
12077 5.0
12079 6.0
12086 15027.0
12093 103.0
12123 8.0
12125 11.0
13019 22.0
13063 587.0
13095 77.0
13099 13.0
13109 7.0
13215 356.0
13261 20.0
13267 25.0
13301 4.0
13303 17.0
16065 205.0
20057 59.0
21013 70.0
21045 22.0
21065 21.0
21075 6.0
21109 15.0
21121 117.0
21129 38.0
21203 21.0
24510 2548.0
26085 50.0
28013 12.0
29223 8.0
30035 10.0
31043 47.0
31047 35.0
34011 451.0
34017 4857.0
34031 3650.0
34039 3462.0
35005 46.0
35019 11.0
35023 4.0
35033 0.0
35045 134.0
35051 8.0
35053 12.0
36047 998.0
36081 1086.0
36087 2028.0
37083 117.0
38005 4.0
40001 6.0
40139 17.0
41049 8.0
45009 18.0
45033 105.0
45053 30.

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 2433
36061 4157
36047 8971 8971
36081 9076
36005 5050


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 96.0, 1003.0: 201.0, 1005.0: 16.0, 1007.0: 17.0, 1009.0: 73.0, 1011.0: 13.0, 1013.0: 35.0, 1015.0: 78.0, 1017.0: 23.0, 1019.0: 18.0, 1021.0: 50.0, 1023.0: 3.0, 1025.0: 10.0, 1027.0: 6.0, 1029.0: 12.0, 1031.0: 49.0, 1033.0: 58.0, 1035.0: 5.0, 1037.0: 9.0, 1039.0: 19.0, 1041.0: 13.0, 1043.0: 111.0, 1045.0: 44.0, 1047.0: 16.0, 1049.0: 38.0, 1051.0: 158.0, 1053.0: 27.0, 1055.0: 53.0, 1057.0: 20.0, 1059.0: 11.0, 1061.0: 27.0, 1063.0: 9.0, 1065.0: 21.0, 1067.0: 17.0, 1069.0: 97.0, 1071.0: 39.0, 1073.0: 881.0, 1075.0: 21.0, 1077.0: 129.0, 1079.0: 23.0, 1081.0: 170.0, 1083.0: 94.0, 1085.0: 8.0, 1087.0: 23.0, 1089.0: 377.0, 1091.0: 19.0, 1093.0: 24.0, 1095.0: 79.0, 1097.0: 373.0, 1099.0: 17.0, 1101.0: 399.0, 1103.0: 84.0, 1105.0: 6.0, 1107.0: 12.0, 1109.0: 52.0, 1111.0: 25.0, 1113.0: 49.0, 1115.0: 95.0, 1117.0: 308.0, 1119.0: 8.0, 1121.0: 152.0, 1123.0: 46.0, 1125.0: 285.0, 1127.0: 57.0, 1129.0: 12.0, 1131.0: 9.0, 1133.0: 16.0, 2013.0: 0.0, 2016.0: 9.0, 2020.0: 1003.0, 2050.0: 108.0, 2

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 954438
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 73.0 38921 187.55941522571362
1001 99.0 55869 177.20023626698168
1009 132.0 57826 228.27101995642101
1013 66.0 19448 339.3665158371041
1015 310.0 113605 272.8753135865499
1017 119.0 33254 357.85168701509593
1031 111.0 52342 212.06679148676014
1033 126.0 55241 228.0914538114806
1039 111.0 37049 299.6032281573052
1043 186.0 83768 222.04182981568138
1045 109.0 49172 221.67086960058572
1051 202.0 81209 248.74090310187296
1055 344.0 102268 336.3711033754449
1067 43.0 17205 249.92734670154027
1069 275.0 105882 259.7230879658488
1071 107.0 51626 207.25990779839614
1077 223.0 92729 240.48571644253684
1079 94.0 32924 285.506013850079
1083 146.0 98915 147.60147601476015
1089 493.0 372909 132.203835252032
1095 217.0 96774 224.23378180089694
1097 792.0 413210 191.67009510902446
1103 265.0 119679 221.42564693889486
1111 41.0 22722 180.4418625121028
1113 37.0 57961 63.83602767378064
1115 234.0 89512 261.41746358030207
1117 235.0 217702 107.94572397130021
1121 168.0 79978 210.05776588561855
226

27093 37.0 23222 159.33166824562915
27097 56.0 33386 167.7349787336009
27107 9.0 6375 141.1764705882353
27111 72.0 58746 122.56153610458584
27115 20.0 29579 67.61553805064403
27121 6.0 11249 53.33807449551071
27125 6.0 4055 147.96547472256475
27127 35.0 15170 230.71852340145026
27129 43.0 14548 295.57327467693153
27133 18.0 9315 193.23671497584542
27135 18.0 15165 118.69436201780415
27137 289.0 199070 145.1750640478224
27143 10.0 14865 67.27211570803902
27147 12.0 36649 32.74304892357227
27151 18.0 9266 194.25857975393913
27153 30.0 24664 121.63477132662992
27157 3.0 21627 13.871549452073797
27161 20.0 18612 107.4575542660649
27163 272.0 262440 103.64273738759336
27167 12.0 6207 193.33011116481393
27173 18.0 9709 185.39499433515294
28003 63.0 36953 170.48683462777043
28033 248.0 184945 134.09391981399875
28045 84.0 47632 176.35203224722875
28057 76.0 23390 324.92518170158183
28059 242.0 143617 168.50372866721906
28081 169.0 85436 197.8088861838101
28089 208.0 106272 195.72417946401686


37097 203.0 181806 111.65748105123043
37101 210.0 209339 100.31575578368101
37109 80.0 86111 92.90334568173637
37115 40.0 21755 183.86577798207307
37121 16.0 14964 106.92328254477412
37123 91.0 27173 334.89125234607883
37125 181.0 100880 179.42109436954797
37127 174.0 94298 184.52141084646544
37139 79.0 39824 198.37284049819206
37141 69.0 63060 109.41960038058991
37145 67.0 39490 169.66320587490503
37151 215.0 143667 149.6516249382252
37157 84.0 91010 92.297549719811
37159 297.0 142088 209.0253927143742
37161 204.0 67029 304.34588014143134
37167 112.0 62806 178.326911441582
37169 78.0 45591 171.0863986313088
37171 155.0 71783 215.92856247300892
37197 47.0 37667 124.77765683489527
38001 3.0 2216 135.37906137184115
38003 31.0 10415 297.6476236197792
38007 0.0 928 0.0
38009 20.0 6282 318.3699458771092
38011 5.0 3024 165.34391534391534
38021 32.0 4872 656.8144499178982
38025 7.0 4424 158.2278481012658
38027 6.0 2287 262.35242675994755
38031 19.0 3210 591.9003115264798
38033 2.0 1761 113.57

51197 75.0 28684 261.4698089527263
51199 50.0 68280 73.22788517867605
51550 275.0 244835 112.32054240611023
51580 12.0 5538 216.68472372697724
51735 16.0 12271 130.38872137560102
51800 178.0 92108 193.25140052981283
51810 364.0 449974 80.89356273918048
51820 34.0 22630 150.24304021210784
53003 28.0 22582 123.99256044637322
53011 248.0 488241 50.79458709940377
53015 77.0 110593 69.62465978859422
53023 5.0 2225 224.71910112359552
53027 57.0 75061 75.93823690065413
53029 28.0 85141 32.8866233659459
53035 95.0 271473 34.99427199021634
53037 31.0 47935 64.67090852195682
53041 55.0 80707 68.14774430966335
53045 27.0 66768 40.438533429187636
53059 1.0 12083 8.276090374906893
53065 28.0 45723 61.23832644402161
53067 84.0 290536 28.912079742269462
54003 119.0 119171 99.85650871436843
54009 60.0 21939 273.4855736359907
54023 27.0 11568 233.40248962655602
54025 63.0 34662 181.75523628180716
54029 97.0 28810 336.6886497743839
54031 24.0 13776 174.21602787456445
54033 79.0 67256 117.46163911026525


20153 7.0 2530 276.6798418972332
20157 15.0 4636 323.5547886108714
20171 21.0 4823 435.41364296081275
20177 352.0 176875 199.01060070671377
20183 2.0 3583 55.81914596706671
20187 2.0 2006 99.70089730807577
20203 0.0 2119 0.0
20205 14.0 8525 164.22287390029325
20207 0.0 3138 0.0
21001 53.0 19202 276.0129153213207
21019 69.0 46718 147.69467871056125
21021 66.0 30060 219.56087824351297
21027 13.0 20477 63.48586218684378
21031 24.0 12879 186.34987188446308
21033 29.0 12747 227.50451086530165
21035 47.0 39001 120.50973051973025
21037 76.0 93584 81.21046332706446
21057 9.0 6614 136.0749924402782
21061 21.0 12150 172.83950617283952
21067 286.0 323152 88.50324305589939
21077 25.0 8869 281.8807080843387
21081 16.0 25069 63.823846184530694
21085 51.0 26427 192.98444772391872
21105 10.0 4380 228.31050228310502
21111 1096.0 766757 142.93967971599866
21113 60.0 54115 110.87498845052204
21137 45.0 24549 183.30685567640228
21139 20.0 9194 217.53317380900586
21155 36.0 19273 186.78980957816634
21167 4

47105 68.0 54068 125.76755197159133
47109 54.0 25694 210.1657974624426
47123 95.0 46545 204.10355569878612
47125 221.0 208993 105.74516849846646
47153 29.0 15026 192.99880207640092
47169 22.0 11284 194.96632399858206
47187 213.0 238412 89.34114054661678
48007 37.0 23510 157.3798383666525
48035 33.0 18685 176.6122558201766
48037 206.0 93245 220.92337390744814
48045 5.0 1546 323.41526520051747
48071 47.0 43837 107.21536601501015
48093 51.0 13635 374.03740374037403
48121 727.0 887207 81.94254553897794
48125 10.0 2211 452.2840343735866
48129 15.0 3278 457.5960951799878
48133 48.0 18360 261.437908496732
48147 107.0 35514 301.28963225770116
48161 48.0 19717 243.44474311507835
48167 392.0 342139 114.57331669292306
48185 66.0 28880 228.53185595567868
48199 108.0 57602 187.4934898093816
48209 236.0 230191 102.52355652479896
48213 179.0 82737 216.34818763068517
48217 104.0 36649 283.7730906709596
48231 164.0 98594 166.33872243747084
48237 16.0 8935 179.0710688304421
48239 31.0 14760 210.02710027

20161 36.0 74232 48.49660523763337
20189 7.0 5485 127.62078395624431
21015 120.0 133581 89.83313495182698
21053 30.0 10218 293.5995302407516
21069 20.0 14581 137.16480351141897
21087 15.0 10941 137.09898546750756
21201 14.0 2108 664.1366223908918
21217 42.0 25769 162.98653420776904
21223 7.0 8471 82.63487191594852
22001 190.0 62045 306.2293496655653
22003 91.0 25627 355.0942365473914
22007 35.0 21891 159.88305696404916
22033 773.0 440059 175.65826400550836
22037 112.0 19135 585.3148680428534
22047 97.0 32511 298.36055488911444
22051 881.0 432493 203.7027188879358
22059 36.0 14892 241.74053182917004
22075 26.0 23197 112.08345906798293
22077 72.0 21730 331.33916244822825
22085 58.0 23884 242.8403952436778
22095 137.0 42837 319.81698064757103
22119 109.0 38340 284.2983828899322
22127 43.0 13904 309.26352128883775
23021 4.0 16785 23.830801310694074
24001 204.0 70416 289.70688479890936
24019 48.0 31929 150.33355256976415
24047 95.0 52276 181.72775269722243
25017 3661.0 1611699 227.151595924

54021 8.0 7823 102.262559120542
54045 86.0 32019 268.5905243761517
54087 8.0 13688 58.44535359438924
54101 7.0 8114 86.27064333251171
55079 1307.0 945726 138.20070506679525
1023 24.0 12589 190.64262451346414
1035 27.0 12067 223.75072511809066
1091 58.0 18863 307.48025234586225
1099 40.0 20733 192.9291467708484
2282 0 579 0.0
5039 15.0 7009 214.01055785418748
5041 24.0 11361 211.2490097702667
5069 163.0 66824 243.92433856099603
5079 48.0 13024 368.55036855036855
5093 106.0 40651 260.7561929595828
5113 71.0 19964 355.6401522740934
5133 24.0 17007 141.1183630269889
6001 1425.0 1671329 85.26148950924684
6013 771.0 1153526 66.83854546841596
6037 23235.0 10039107 231.44488847464223
6053 342.0 434061 78.79076903937465
6059 4761.0 3175692 149.9200804108207
6067 1613.0 1552058 103.92652851890846
6081 553.0 766573 72.13924831685958
6083 443.0 446499 99.21634762899805
6085 1950.0 1927852 101.1488433759438
6101 106.0 96971 109.31103113301914
6115 44.0 78668 55.93125540245081
8001 641.0 517421 123.

48201 5857.0 4713325 124.26471758259827
48207 27.0 5658 477.20042417815483
48249 95.0 40482 234.67219999011905
48255 36.0 15601 230.75443881802448
48271 6.0 3667 163.6214889555495
48279 81.0 12893 628.2478864500117
48301 0.0 169 0.0
48341 71.0 20940 339.0639923591213
48357 26.0 9836 264.33509556730377
48369 35.0 9605 364.3935450286309
48371 34.0 15823 214.87707767174368
48375 433.0 117415 368.77741344802627
48389 41.0 15976 256.634952428643
48419 66.0 25274 261.13792830576875
48435 8.0 3776 211.864406779661
48437 17.0 7397 229.8229011761525
48463 70.0 26741 261.7703152462511
48465 212.0 49025 432.43243243243245
48501 27.0 8713 309.8817858372547
51027 36.0 21004 171.39592458579318
51051 15.0 14318 104.76323508869955
51105 45.0 23423 192.11885753319388
51183 21.0 11159 188.18890581593334
51195 92.0 37383 246.10116898055264
51640 50.0 6347 787.7737513786041
51710 235.0 242742 96.81060549884239
51720 6.0 3981 150.71590052750565
54005 38.0 21457 177.0983828121359
54015 9.0 8508 105.78279266

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2002.0, 854757, 234.21861417923458)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 6651.073050039002,
 'ALABAMA': 17526.705968158658,
 'ALASKA': 658.5884275736595,
 'ARIZONA': 4496.125639293312,
 'ARKANSAS': 16515.516651316786,
 'CALIFORNIA': 5645.175157819054,
 'COLORADO': 7236.694637566467,
 'CONNECTICUT': 1535.106088642782,
 'DELAWARE': 515.2591106657457,
 'FLORIDA': 12224.791368233575,
 'GEORGIA': 40748.7048151141,
 'IDAHO': 4820.412358006737,
 'ILLINOIS': 22348.592415262352,
 'INDIANA': 18998.787273458493,
 'IOWA': 23841.549397740302,
 'KANSAS': 17961.82559719784,
 'KENTUCKY': 19342.542684783075,
 'LOUISIANA': 16691.87961960506,
 'MAINE': 760.912236972799,
 'MARYLAND': 3320.9619175874636,
 'MASSACHUSETTS': 2875.259737079572,
 'MICHIGAN': 13237.492960577003,
 'MINNESOTA': 11857.026026554075,
 'MISSISSIPPI': 23574.320168457918,
 'MISSOURI': 16597.959827944353,
 'MONTANA': 10691.812895981715,
 'NEBRASKA': 13173.372351222997,
 'NEVADA': 2098.0227008020443,
 'NEW HAMPSHIRE': 759.9963493854957,
 'NEW JERSEY': 5390.867421456342,
 'NEW YORK': 8768.6963766

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 954437.9999999999, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76274734.0 1000000 10122.644282181305
ALABAMA 954437.9999999999 76274734.0 1000000 12513.160648977155
ALASKA 231145.0 76274734.0 1000000 3030.4268252184265
ARIZONA 1839932.0 76274734.0 1000000 24122.43089566199
ARKANSAS 830467.0000000003 76274734.0 1000000 10887.838691118874
CALIFORNIA 11847711.0 76274734.0 1000000 155329.43058182282
COLORADO 1337805.0 76274734.0 1000000 17539.294204552716
CONNECTICUT 874974.0 76274734.0 1000000 11471.347772907344
DELAWARE 239009.0 76274734.0 1000000 3133.527807517493
FLORIDA 3892551.999999999 76274734.0 1000000 51033.30809387023
GEORGIA 1928702.999999999 76274734.0 1000000 25286.263207420678
IDAHO 340742.0000000001 76274734.0 1000000 4467.298437251845
ILLINOIS 2987496.0000000005 76274734.0 1000000 39167.5702205661
INDIANA 1602975.9999999995 76274734.0 1000000 21015.818947333195
IOWA 699741.0 76274734.0 1000000 9173.955296913917
KANSAS 401103.0000000001 76274734.0 1000000 5258.661406803465
KENTUCKY 1416013.0000000002 76274734.0 1000

In [98]:
print (Covid_state)

{'NEW MEXICO': 2603.0, 'ALABAMA': 5345.0, 'ALASKA': 2282.0, 'ARIZONA': 7486.0, 'ARKANSAS': 2619.0, 'CALIFORNIA': 34667.0, 'COLORADO': 15844.0, 'CONNECTICUT': 17205.0, 'DELAWARE': 3644.0, 'FLORIDA': 66860.0, 'GEORGIA': 19869.0, 'IDAHO': 3807.0, 'ILLINOIS': 31024.0, 'INDIANA': 11834.0, 'IOWA': 7078.0, 'KANSAS': 2955.0, 'KENTUCKY': 7985.0, 'LOUISIANA': 6056.0, 'MAINE': 2895.0, 'MARYLAND': 15376.0, 'MASSACHUSETTS': 24908.0, 'MICHIGAN': 68361.0, 'MINNESOTA': 19928.0, 'MISSISSIPPI': 3055.0, 'MISSOURI': 7903.0, 'MONTANA': 1992.0, 'NEBRASKA': 4988.0, 'NEVADA': 3506.0, 'NEW HAMPSHIRE': 4587.0, 'NEW JERSEY': 56474.0, 'NEW YORK': 79412.0, 'NORTH CAROLINA': 21611.0, 'NORTH DAKOTA': 1880.0, 'OHIO': 22904.0, 'OKLAHOMA': 5072.0, 'OREGON': 4673.0, 'PENNSYLVANIA': 49033.0, 'RHODE ISLAND': 4345.0, 'SOUTH CAROLINA': 13692.0, 'SOUTH DAKOTA': 2500.0, 'TENNESSEE': 15178.0, 'TEXAS': 47888.0, 'UTAH': 5570.0, 'VERMONT': 2199.0, 'VIRGINIA': 18727.0, 'WASHINGTON': 12020.0, 'WEST VIRGINIA': 5194.0, 'WISCONSIN': 9

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 6651.073050039002,
 'ALABAMA': 17526.705968158658,
 'ALASKA': 658.5884275736595,
 'ARIZONA': 4496.125639293312,
 'ARKANSAS': 16515.516651316786,
 'CALIFORNIA': 5645.175157819054,
 'COLORADO': 7236.694637566467,
 'CONNECTICUT': 1535.106088642782,
 'DELAWARE': 515.2591106657457,
 'FLORIDA': 12224.791368233575,
 'GEORGIA': 40748.7048151141,
 'IDAHO': 4820.412358006737,
 'ILLINOIS': 22348.592415262352,
 'INDIANA': 18998.787273458493,
 'IOWA': 23841.549397740302,
 'KANSAS': 17961.82559719784,
 'KENTUCKY': 19342.542684783075,
 'LOUISIANA': 16691.87961960506,
 'MAINE': 760.912236972799,
 'MARYLAND': 3320.9619175874636,
 'MASSACHUSETTS': 2875.259737079572,
 'MICHIGAN': 13237.492960577003,
 'MINNESOTA': 11857.026026554075,
 'MISSISSIPPI': 23574.320168457918,
 'MISSOURI': 16597.959827944353,
 'MONTANA': 10691.812895981715,
 'NEBRASKA': 13173.372351222997,
 'NEVADA': 2098.0227008020443,
 'NEW HAMPSHIRE': 759.9963493854957,
 'NEW JERSEY': 5390.867421456342,
 'NEW YORK': 8768.6963766

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)